Imports

In [8]:
import feed_forward
import numpy as np
import torch
import torch.nn as nn
import geoopt

Initializing the model

In [9]:
ball = geoopt.PoincareBall()
model = feed_forward.HypFF()
params = list(model.parameters())
point = torch.randn(1)
input = ball.projx(point)
output = model(input)

In [10]:
print(model)
print("Output is :", output)

HypFF(
  (fc1): MobLinear(
    in_features=1, out_features=1, bias=True
    (ball): PoincareBall manifold
  )
  (fc2): MobLinear(
    in_features=1, out_features=1, bias=True
    (ball): PoincareBall manifold
  )
)
Output is : tensor([-0.5963], grad_fn=<SWhereBackward>)


Loss Function

In [13]:
target = torch.randn(1)  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.0993, grad_fn=<MseLossBackward>)
